## C01 - Use CSP embeddings

Simple example of how to obtain pretrained CSP embeddings. Read the paper here:[https://arxiv.org/abs/2305.01118](https://arxiv.org/abs/2305.01118). Note that this notebook needs to be run with GPU enabled. To do this got to: "Runtime -> Change runtime type"

In [1]:
!rm -r sample_data .config # Empty current directory
!git clone https://github.com/gengchenmai/csp.git . # Clone CSP repository

Cloning into '.'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 86 (delta 26), reused 75 (delta 15), pack-reused 0
Receiving objects: 100% (86/86), 1.58 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (26/26), done.


Import required packages.

In [2]:
import numpy as np
import pandas as pd
import torch

import sys
sys.path.append('./main')

from main.utils import *
from main.models import *

Write helper function to load CPS models from checkpoint.

In [3]:
def get_csp(path):
    pretrained_csp = torch.load(path)

    params = pretrained_csp['params']
    loc_enc = get_model(
                            train_locs = None,
                            params = params,
                            spa_enc_type = params['spa_enc_type'],
                            num_inputs = params['num_loc_feats'],
                            num_classes = params['num_classes'],
                            num_filts = params['num_filts'],
                            num_users = params['num_users'],
                            device = params['device'])

    model = LocationImageEncoder(loc_enc = loc_enc,
                        train_loss = params["train_loss"],
                        unsuper_loss = params["unsuper_loss"],
                        cnn_feat_dim = params["cnn_feat_dim"],
                        spa_enc_type = params["spa_enc_type"]).to(params['device'])

    model.load_state_dict(pretrained_csp['state_dict'])

    return model

Download pretrained models. For details see here: [https://gengchenmai.github.io/csp-website/](https://gengchenmai.github.io/csp-website/)

In [4]:
!wget -O model_dir.zip 'https://www.dropbox.com/s/qxr644rj1qxekn2/model_dir.zip?dl=1'

--2024-01-21 01:57:39--  https://www.dropbox.com/s/qxr644rj1qxekn2/model_dir.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.72.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/qxr644rj1qxekn2/model_dir.zip [following]
--2024-01-21 01:57:39--  https://www.dropbox.com/s/dl/qxr644rj1qxekn2/model_dir.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc88d2f6a29caa94bff34c8e0b68.dl.dropboxusercontent.com/cd/0/get/CLsPN3FzQu8_q5uGtPvkDlDROOyJ6xA-YyXdunQrn7KAzziriwxL6D5CtJ-GLtERcoOgOxXfm552bAerCPxoiiK73Gcy-_d58UA0Im4LxotstFW-4wLSnulKjrUuv0l_OrxCIyAXp_GP2OHpWKHcgdQN/file?dl=1# [following]
--2024-01-21 01:57:40--  https://uc88d2f6a29caa94bff34c8e0b68.dl.dropboxusercontent.com/cd/0/get/CLsPN3FzQu8_q5uGtPvkDlDROOyJ6xA-YyXdunQrn7KAzziriwxL6D5CtJ-GLtERcoOgOxXfm552bAerCPxoiiK73Gcy-_

In [5]:
!unzip model_dir.zip

Archive:  model_dir.zip
   creating: model_dir/
  inflating: __MACOSX/._model_dir    
  inflating: model_dir/.DS_Store     
  inflating: __MACOSX/model_dir/._.DS_Store  
   creating: model_dir/model_inat_2018/
  inflating: __MACOSX/model_dir/._model_inat_2018  
   creating: model_dir/model_fmow/
  inflating: __MACOSX/model_dir/._model_fmow  
  inflating: model_dir/model_inat_2018/.DS_Store  
  inflating: __MACOSX/model_dir/model_inat_2018/._.DS_Store  
  inflating: model_dir/model_inat_2018/model_inat_2018_gridcell_0.0010_32_0.1000000_1_512_leakyrelu_contsoftmax_ratio0.050_0.000500_1.000_1_1.000_TMP20.0000_1.0000_1.0000.pth.tar  
  inflating: __MACOSX/model_dir/model_inat_2018/._model_inat_2018_gridcell_0.0010_32_0.1000000_1_512_leakyrelu_contsoftmax_ratio0.050_0.000500_1.000_1_1.000_TMP20.0000_1.0000_1.0000.pth.tar  
  inflating: model_dir/model_inat_2018/model_inat_2018_gridcell_0.0010_32_0.1000000_1_512_leakyrelu_UNSUPER-contsoftmax_0.000500_1.000_1_1.000_TMP20.0000_1.0000_1.0000.pt

Load CSP model.

In [6]:
path = '/content/model_dir/model_fmow/model_fmow_gridcell_0.0010_32_0.1000000_1_512_gelu_UNSUPER-contsoftmax_0.000050_1.000_1_0.100_TMP1.0000_1.0000_1.0000.pth.tar'
model = get_csp(path)

/content/./main/module.py:98: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.linear.weight)


Use CSP model to obtain location embeddings.

In [7]:
c = torch.randn(32, 2)  # Represents a batch of 32 locations (lon/lat)

model.eval()
with torch.no_grad():
    emb = model.loc_enc(convert_loc_to_tensor(c.numpy()),return_feats=True).detach().cpu()

In [8]:
emb.shape

torch.Size([32, 256])